In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from tensorboardX import SummaryWriter      

import torchvision
import torchvision.transforms as transforms

from models import *

global best_prec
use_gpu = torch.cuda.is_available()
# print('=> Building model...')
    
    
batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()
print(model)
# device = torch.device("cuda") 
# model = VGG16_quant().to(device)
# if torch.cuda.device_count() > 1:
#     print(f"=> Using {torch.cuda.device_count()} GPUs")
#     model = nn.DataParallel(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [2]:
PATH = "result/VGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9295/10000 (93%)



In [3]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    # print(layer)
    i = i+1
    if isinstance(layer, QuantConv2d):
        # print(layer)
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

dataiter = iter(testloader)
images, labels = next(dataiter)
images = images.to(device)
out = model(images)

3 -th layer prehooked
7 -th layer prehooked
12 -th layer prehooked
16 -th layer prehooked
21 -th layer prehooked
25 -th layer prehooked
29 -th layer prehooked
34 -th layer prehooked
38 -th layer prehooked
41 -th layer prehooked
46 -th layer prehooked
50 -th layer prehooked
54 -th layer prehooked


In [4]:
weight_q = model.features[27].weight_q
w_alpha = model.features[27].weight_quant.wgt_alpha

# print(weight_q)
w_bit = 4

weight_int = weight_q / (w_alpha / (2**(w_bit-1)-1))
# print(weight_int)

In [5]:
k = 8  # input to your 8→8 conv layer

act = save_output.outputs[8][0]      # [128, 8, 4, 4]
act_alpha = model.features[27].act_alpha
act_bit = 4
act_quant_fn = act_quantization(act_bit)

# quantize + convert to integer
act_q   = act_quant_fn(act, act_alpha)
act_int = act_q / (act_alpha / (2**act_bit - 1))

# sanity check
print(act_int.shape)  # should be [128, 8, 4, 4]


torch.Size([128, 8, 4, 4])


In [6]:
conv_int = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, padding=1)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
conv_int.bias = model.features[27].bias
output_int = conv_int(act_int)
output_recovered = output_int * (act_alpha / (2**act_bit-1)) * (w_alpha / (2**(w_bit-1)-1))
# print(output_recovered)

In [7]:
device = act.device  # should be 'cuda:0'

model = model.to(device)

with torch.no_grad():
    conv_ref = model.features[27]   # already on cuda with the model
    output_ref = conv_ref(act)      # act is on same device

print((output_ref - output_recovered).abs().mean())

tensor(5.6042e-08, device='cuda:0', grad_fn=<MeanBackward0>)


In [8]:
# act_int.size = torch.Size([128, 64, 32, 32])  <- batch_size, input_ch, ni, nj
a_int = act_int[0,:,:,:]  # pick only one input out of batch

w_int = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))  # merge ki, kj index to kij
 # weight_int: [out_ch, in_ch, ki, kj]  ->  [out_ch, in_ch, kij]
print("w_int shape:", w_int.shape)   # should be [8, 8, 9]
# print(a_int.shape)
padding = 1
stride = 1
array_size = 8 # row and column number

nig = range(a_int.size(1))  ## ni group
njg = range(a_int.size(2))  ## nj group

icg = range(int(w_int.size(1)))  ## input channel 
ocg = range(int(w_int.size(0)))  ## output channel

ic_tileg = range(int(len(icg)/array_size))
oc_tileg = range(int(len(ocg)/array_size))

kijg = range(w_int.size(2))
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size

######## Padding before Convolution #######
a_pad = torch.zeros(len(icg), len(nig)+padding*2, len(nig)+padding*2).cuda()
# a_pad.size() = [64, 32+2pad, 32+2pad]
a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = a_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))
# a_pad.size() = [64, (32+2pad)*(32+2pad)]


a_tile = torch.zeros(len(ic_tileg), array_size,    a_pad.size(1)).cuda() 
w_tile = torch.zeros(len(oc_tileg)*len(ic_tileg), array_size, array_size, len(kijg)).cuda() 

for ic_tile in ic_tileg:
    a_tile[ic_tile,:,:] = a_pad[ic_tile*array_size:(ic_tile+1)*array_size,:]

for ic_tile in ic_tileg:
    for oc_tile in oc_tileg:
        w_tile[oc_tile*len(oc_tileg) + ic_tile,:,:,:] = w_int[oc_tile*array_size:(oc_tile+1)*array_size, ic_tile*array_size:(ic_tile+1)*array_size, :]



###########################################

p_nijg = range(a_pad.size(1)) ## psum nij group

psum = torch.zeros(len(ic_tileg), len(oc_tileg), array_size, len(p_nijg), len(kijg)).cuda() 

for kij in kijg:
    for ic_tile in ic_tileg:       # Tiling into array_sizeXarray_size array
        for oc_tile in oc_tileg:   # Tiling into array_sizeXarray_size array        
            for nij in p_nijg:       # time domain, sequentially given input
                    m = nn.Linear(array_size, array_size, bias=False)
                    #m.weight = torch.nn.Parameter(w_int[oc_tile*array_size:(oc_tile+1)*array_size, ic_tile*array_size:(ic_tile+1)*array_size, kij])
                    m.weight = torch.nn.Parameter(w_tile[len(oc_tileg)*oc_tile+ic_tile,:,:,kij])
                    psum[ic_tile, oc_tile, :, nij, kij] = m(a_tile[ic_tile,:,nij]).cuda()
 

w_int shape: torch.Size([8, 8, 9])


In [10]:
'''
activation.txt code
'''
############################################################
# 6x6 activation.txt for HW TB (36 nij, 8ch, 4 bits each)
# - assumes act_int is already defined:
#   act_int.shape == [batch, 8, 4, 4]
#   (from layer 27 prehook in proj_kij notebook)
############################################################

act_bits = 4

# take one sample from the batch
a_core = act_int[0]        # [8, 4, 4]
C, H, W = a_core.shape     # C=8, H=W=4

# target spatial size for HW tile
Hpad, Wpad = 6, 6

# zero-pad to 6x6, with 4x4 centered
a_padded = torch.zeros(C, Hpad, Wpad, dtype=a_core.dtype, device=a_core.device)
a_padded[:, 1:1+H, 1:1+W] = a_core  # put 4x4 in the middle

print("a_core shape   :", a_core.shape)    # [8, 4, 4]
print("a_padded shape :", a_padded.shape)  # [8, 6, 6] ← 36 nij

# ---------- write activation.txt ----------
with open("activation.txt", "w") as f:
    # ni, nj now run over 6x6 = 36 nij
    for i in range(Hpad):        # rows 0..5
        for j in range(Wpad):    # cols 0..5
            vals = []
            for ch in range(C):  # 8 channels
                v = int(round(a_padded[ch, i, j].item()))
                # wrap negatives into 4-bit unsigned two’s-complement style
                if v < 0:
                    v += (1 << act_bits)
                vals.append(v & ((1 << act_bits) - 1))

            # Pack ch0..ch7 into 32 bits (MSB = ch0’s MSB)
            word = 0
            for v in vals:
                word = (word << act_bits) | v
            bits32 = f"{word:032b}"
            f.write(bits32 + "\n")

print("Wrote 36 nij (6x6) activations to activation.txt")

a_core shape   : torch.Size([8, 4, 4])
a_padded shape : torch.Size([8, 6, 6])
Wrote 36 nij (6x6) activations to activation.txt


In [30]:
import torch
import torch.nn.functional as F

# ===== Choose layer-27 tile =====
# act_int shape at layer 27: [batch, 8, 4, 4]
# weight_int shape: [8, 8, 3, 3]
a_full = act_int[0, :8, :, :]        # [8, 4, 4]
w_full = weight_int[:8, :8, :, :]    # [8, 8, 3, 3]

# Use the whole 4x4 as the tile
a_tile = a_full                      # [8, 4, 4]

# Conv wants [N, C, H, W]
a_tile_batch = a_tile.unsqueeze(0).float()   # [1, 8, 4, 4]
w_full_f     = w_full.float()               # [8, 8, 3, 3]

# ===== Conv: padding=1, stride=1 → 4x4 output =====
tile_out = F.conv2d(
    a_tile_batch,
    w_full_f,
    bias=None,
    stride=1,
    padding=1,    # keep spatial size 4x4
)   # [1, 8, 4, 4]

# ===== ReLU =====
tile_out_relu = torch.clamp(tile_out, min=0.0)   # [1, 8, 4, 4]

# ===== Convert to 16-bit integer psums =====
tile_out_int = torch.round(tile_out_relu).to(torch.int16)  # [1, 8, 4, 4]
tile_out_int = tile_out_int[0]                              # [8, 4, 4]

print("tile_out_int shape:", tile_out_int.shape)  # -> [8, 4, 4]

# ===== Write to txt: each line = 128 bits (8 ch x 16 bits) =====
out_path = "output_relu_tile.txt"

with open(out_path, "w") as f:
    for h in range(tile_out_int.size(1)):     # 4 rows
        for w in range(tile_out_int.size(2)): # 4 cols
            vals = tile_out_int[:, h, w]      # [8]

            bits_per_pixel = []
            for v in vals:
                v16   = int(v.item()) & 0xFFFF
                bits16 = format(v16, "016b")  # 16-bit binary, MSB first
                bits_per_pixel.append(bits16)

            line = "".join(bits_per_pixel)    # 8 * 16 = 128 bits
            f.write(line + "\n")

print("Wrote ReLU outputs to", out_path)

tile_out_int shape: torch.Size([8, 4, 4])
Wrote ReLU outputs to output_relu_tile.txt


In [31]:
'''
Generate weight_kij files for layer 27
weight_k0.txt ... weight_k8.txt
Each file: 8 rows (oc0..oc7), each row = 8 weights (4-bit two's complement) packed into 32 bits
'''

import os

w_bit = 4

# weight_int for layer 27 should be [8, 8, 3, 3]
# if it's larger, slice first 8x8
w27_int = weight_int[:8, :8, :, :]          # [8, 8, 3, 3]
w_int   = w27_int.view(8, 8, -1)            # [8, 8, 9]  -> merge ki,kj into kij

os.makedirs("weight_kij", exist_ok=True)

num_kij = w_int.size(2)   # 9
print("num_kij =", num_kij)

for kij in range(num_kij):
    fname = f"weight_kij/weight_k{kij}.txt"
    with open(fname, "w") as f:
        # For each output channel (column)
        for oc in range(w_int.size(0)):   # 0..7
            vals = []

            # 8 input channels -> 8 x 4 bits
            # iC7..iC0 -> MSB..LSB
            for r in range(w_int.size(1)):    # 0..7
                ic = w_int.size(1) - 1 - r    # reverse so ic7 is written first
                v = int(round(w_int[oc, ic, kij].item()))
                if v < 0:
                    v += (1 << w_bit)
                v &= (1 << w_bit) - 1         # keep 4 bits
                vals.append(v)

            # Pack into 32-bit word
            word = 0
            for v in vals:                    # vals[0] = ic7 ... vals[7] = ic0
                word = (word << w_bit) | v

            bits32 = f"{word:032b}"
            f.write(bits32 + "\n")

    print(f"Saved {fname}")


num_kij = 9
Saved weight_kij/weight_k0.txt
Saved weight_kij/weight_k1.txt
Saved weight_kij/weight_k2.txt
Saved weight_kij/weight_k3.txt
Saved weight_kij/weight_k4.txt
Saved weight_kij/weight_k5.txt
Saved weight_kij/weight_k6.txt
Saved weight_kij/weight_k7.txt
Saved weight_kij/weight_k8.txt


In [20]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# ---------- 1. Quantization parameters ----------
w_bit   = 4
act_bit = 4

# weights for layer 27
weight_q  = model.features[27].weight_q.detach()              # [8, 8, 3, 3]
w_alpha   = model.features[27].weight_quant.wgt_alpha.detach()
w_delta   = w_alpha / (2**(w_bit-1) - 1)                      # scalar
w_int     = weight_q / w_delta                                # [8, 8, 3, 3]

# activations for layer 27 (input to layer 27)
act_alpha    = model.features[27].act_alpha.detach()
act_quant_fn = act_quantization(act_bit)

# x27: float input to layer 27, shape [1, 8, 4, 4]
# assumes `act_int` was computed earlier from the same hook:
x27 = act_int[0].unsqueeze(0)                                 # [1, 8, 4, 4]

x_q    = act_quant_fn(x27, act_alpha)                         # [1, 8, 4, 4]
a_delta = act_alpha / (2**act_bit - 1)
a_int   = x_q / a_delta                                       # [1, 8, 4, 4]

print("a_int shape:", a_int.shape)
print("w_int shape:", w_int.shape)

# ---------- 2. Conv2d with integer weights ----------
conv2 = nn.Conv2d(
    in_channels = 8,
    out_channels = 8,
    kernel_size = 3,
    padding = 1,
    bias = False,
).to(device)

conv2.weight = nn.Parameter(w_int.float().to(device))

# integer psum (computed as float), then ReLU
with torch.no_grad():
    psum_int = conv2(a_int.float().to(device))                # [1, 8, 4, 4]
    psum_int = torch.clamp(psum_int, min=0)
    psum_recovered = psum_int * w_delta * a_delta             # [1, 8, 4, 4]

print("psum_recovered shape:", psum_recovered.shape)

# ---------- 3. True reference: Conv27 + ReLU28 ----------
with torch.no_grad():
    y = model.features[27](x27.to(device))                    # QuantConv2d 8->8
    y = model.features[28](y)                                 # ReLU
    y_ref = y[0]                                              # [8, 4, 4]

# ---------- 3b. Get act29_in from saved prehook outputs ----------
# save_output.outputs[k] stores the input to each QuantConv2d (in order).
# Earlier you used k = 8 for layer 27, so layer 29 is the next conv: k = 9.
act29_batch = save_output.outputs[9][0]              # [batch, 8, 4, 4]
act29_in    = act29_batch[0]                                  # [8, 4, 4]

# ---------- 4. Compare errors ----------
err_ref  = (psum_recovered[0] - y_ref.to(device)).abs().max()
err_next = (psum_recovered[0] - act29_in.to(device)).abs().max()

print("max |psum_recovered - conv27+relu28|:", err_ref.item())
print("max |psum_recovered - act29_in|    :", err_next.item())


a_int shape: torch.Size([1, 8, 4, 4])
w_int shape: torch.Size([8, 8, 3, 3])
psum_recovered shape: torch.Size([1, 8, 4, 4])
max |psum_recovered - conv27+relu28|: 4.76837158203125e-07
max |psum_recovered - act29_in|    : 1.6319186687469482
